### Threading notebook 
Examples below are taken mainly from [here](https://pymotw.com/2/threading/)

In [ ]:
import threading
import time

A thread that waits for 1 second before printing a number, counting from 1 to 10

In [ ]:
def loop1_10():
    for i in range(1,11):
        time.sleep(1)
        print (i)

In [ ]:
threading.Thread(target = loop1_10).start()

We define a class called `MyThread` that runs for 1 second and then instantiate threads numbered from 1 to 5 at 0.9 seconds interval

In [ ]:
class MyThread(threading.Thread):
    def run(self):
        print ('{} started!'.format(self.getName()))
        time.sleep(1)
        print ('{} finished!'.format(self.getName()))

In [ ]:
for x in range(5):
    mythread = MyThread(name = 'Thread-{}'.format(x + 1))
    mythread.start()
    time.sleep(0.9)

Using threads allows a program to run multiple operations concurrently in the same process space. The simplest way to use a `Thread` is to instantiate it with a target function and call `start()` to let it begin working.

In [ ]:
def worker():
    print ('worker')
    return

threads = []

for i in range(5):
    t = threading.Thread(target = worker)
    threads.append(t)
    t.start()
print (threads)    

It is useful to be able to spawn a thread and pass it arguments to tell it what work to do. The example below passes a number, which the thread then prints.

In [ ]:
def worker(num):
    print ('Worker: %s' %num)
    return

threads = []

for i in range(5):
    t = threading.Thread(target = worker, args = (i,))
    threads.append(t)
    t.start()


#### Determining the current thread

Using arguments to identify or naming the thread is cumbersome and unnecessary. Each `Thread` instance has a name with a default value that can be changed as the thread is created. Naming threads is useful in server processes with multiple service threads handling different operations.

In [ ]:
# import threading
# import time

def worker():
    print (threading.currentThread().getName(), 'Starting')
#     print (threading.current_thread().getName(), 'Starting')
    time.sleep(2)
    print (threading.currentThread().getName(), 'Exiting')

def my_service():
    print (threading.currentThread().getName(), 'Starting')
    time.sleep(3)
    print (threading.currentThread().getName(), 'Exiting')    

In [ ]:
t = threading.Thread(name = 'my_service', target = my_service)
w1 = threading.Thread(name = 'worker', target = worker)
w2 = threading.Thread(target = worker)

t.start()
w1.start()
w2.start()

Most programs do not use **print** to debug. The `logging` module supports embedding the thread name in every log message using the formatter code `%(threadName)s`. Including thread names in log messages makes it easier to trace those messages back to their source.

In [ ]:
import logging

In [ ]:
logging.basicConfig(level = logging.DEBUG, format = '[%(levelname)s] (%(threadName)-10s) %(message)s',)

def worker():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

def my_service():
    logging.debug('Starting')
    time.sleep(3)
    logging.debug('Exiting')    

In [ ]:
t = threading.Thread(name = 'my_service', target = my_service)
w1 = threading.Thread(name = 'worker', target = worker)
w2 = threading.Thread(target = worker)

w1.start()
w2.start()
t.start()
                     

#### Daemon vs Non-Daemon Threads

The example programs above have implicitly waited to exit until all threads have completed their work. Sometimes programs spawn a thread as a *daemon* that runs without blocking the main program from exiting. Using daemon threads is useful for services where there may not be an easy way to interrupt the thread or where letting the thread die in the middle of its work does not lose or corrupt data. To mark a thread as a daemon, call its `setDaemon()` method with a boolean argument. The default is for threads to not be daemons, so passing `True` turns the daemon mode on.

In [ ]:
logging.basicConfig(level = logging.DEBUG, format = '(%(threadName)-10s) %(message)',)

def daemon():
    logging.debug('Starting')
    time.sleep(5)
    logging.debug('Exiting')
    
d = threading.Thread(name = 'daemon', target = daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('Starting')
#     time.sleep(3)
    logging.debug('Exiting')

t = threading.Thread(name = 'non-daemon', target = non_daemon)    

d.start()
t.start()

In the example above, we should not be seeing the last line `[DEBUG] (daemon     ) Exiting` if it was run on command prompt, but somehow the line shows in jupyter notebook

To wait until a daemon thread has completed its work, use the `join()` method

In [ ]:
d = threading.Thread(name = 'daemon', target = daemon)
d.setDaemon(True)

t = threading.Thread(name = 'non-daemon', target = non_daemon)    

d.start()
t.start()

d.join()
t.join()


In [ ]:
d = threading.Thread(name = 'daemon', target = daemon)
d.setDaemon(True)

t = threading.Thread(name = 'non-daemon', target = non_daemon)    

d.start()
t.start()

print ('d.isAlive()', d.isAlive())
d.join(6)
# print ('d.isAlive()', d.isAlive())
# d.join(3)
print ('t.isAlive()', t.isAlive())

t.join()
print ('d.isAlive()', d.isAlive())


#### Enumerating All Threads

It is not necessary to retain an explicit handle to all of the daemon threads in order to ensure they have completed before exiting the main process. `enumerate()` returns a list of active `Thread` instances. The list includes the current thread, and since joining the current thread is not allowed (it introduces a deadlock situation (why? idu)), it must be skipped.

In [ ]:
import random
import threading
import time
import logging

logging.basicConfig(level = logging.DEBUG, format = '(%(threadName)-10s) %(message)s',)

def worker():
    t = threading.currentThread()
    pause = random.randint(1,5)
    logging.debug('sleeping %s' % pause)
    time.sleep(pause)
    logging.debug('ending')
    return

for i in range(3):
    t = threading.Thread(target = worker)
    t.setDaemon(True)
    t.start()

main_thread = threading.currentThread()
for t in threading.enumerate():
    if t is main_thread:
        continue
    logging.debug('joining %s', t.getName())
    t.join()

Threading exercises doesn't seem to work the way they are supposed to work in the jupyter notebook.